# How Data Clustering Works

In this notebook we will consider a Neural Network (NN) approach to clustering in high dimensional data.

Please read through the [How cluster works traditional ML only approach]() notebook before this one as it explains the purpose and basic elements of clustering algorithms.  

### What have we learned so far?

From the first notebook we learned that a family of methods exists called 'clustering' which divides data into categories which display similar properties in some space. That is, they are near to each other under some definition of distance, that we call a metric.   

### Why involve a NN?

Now what if we had a set of data with many dimensions and features and our traditional clustering algorithms don't appear to be working that well...

This is where exploring some unsupervised NN architectures might be really useful.  In particular algorithms which can reduce the dimensionality of the problem can improve clustering algorithms. Instead of trying to group similar objects in a pre-define metric space we use a NN to learn what the key features are in some lower dimensional space and we then do our grouping there.

Before we see an example of this it's worth thinking about dome jargon. You may have come across the term 'embedding' or 'latent space' - what are they?

### Embedding and latent spaces

In popular literature you'll likely see these terms used interchangeably. However, they do have a small subtle difference.

Embedding vectors take high dimensional discrete data and encode it into a lower dimensional vector space. So discrete categorical data is turned into a continuous vector embedding. Now we do use this also with continuous data but where those data are treated like categorical data - this makes sense as we are trying to use an algorithm to cluster, and therefore to place our data into like-minded categories. So our embedding vectors are capturing information about the similarities on the data - sounds similar already to clustering... so an embedding tells us the way the high-dimensional data is mapped to low-dimensional space.

A latent space is most often used to mean the space which could contain the embedding vectors but more generally is a space which captures a compressed representation of the features of any data. However, note that occasionally 'latent space' is used for the space of a hidden (latent) variable, which cannot be observed directly.

In this notebook we will stick to the terminology embeddings.

### The architecture

Essentially we want a mechanism which reduces the dimensions. So instead of a network where we have layers of the same size we will have a network where we reduce the layer size with subsequent layers in the network. This will look something like this:

![autoencoder_pic](https://github.com/reac2/UKSA_SoftwareDataAI_Training/blob/main/UKSA_SoftwareDataAI_Training/Phase2_Materials/static_images/encoder_image.jpg?raw=true)

We will take our inputs, apply a NN where the layers decrease in dimensionality through the layers, and run our clustering algorithm on this learned reduced dimensionality embedding. But how do we actually learn a useful embedding?

### What is an Autoencoder?

To learn a useful embedding we need another side to our NN - a decoder. The encoder encodes the embedding, reducing the dimensionality and then the decoder take this reduced dimensionality space and expands it back to the original dimensions.

![autoencoder_pic](https://github.com/reac2/UKSA_SoftwareDataAI_Training/blob/main/UKSA_SoftwareDataAI_Training/Phase2_Materials/static_images/autoencoder_image.jpg?raw=true)

The algorithm will be successful once it has iterated to find an embedding which, for the given embedding size, maximizes the accuracy of the output matching the input. There are therefore no labels needed as the algorithm inputs and 'truth' of the outputs are equal to each other.

When the accuracy is high the embedding space must be encoding the 'most useful' information for reconstructing the image, therefore we can see that this space will be a much more efficient one in which to run our clustering algorithm.

### So let's give it a go!

We'll use a very standard dataset for this example: the MNIST handwritten digits dataset.  

This code not mine - must rewrite and want to rewrite to make more understandable.

In [2]:
# Import Numpy for arrays
import numpy as np

# Import our PyTorch libraries
import torch
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader, ConcatDataset

# We will use the MNIST dataset
from torchvision.datasets import MNIST
from torchvision import transforms

# Scikit-learn provides metrics to measure the performance of our models.
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score

We are going to use the [MNIST database of handwritten digits](https://yann.lecun.com/exdb/mnist/). We are going to merge the training and test datasets and set up a [Dataloader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders).

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = MNIST('./', download=True,
                 train=True,
                 transform=transform)
testset = MNIST('./', download=True,
                 train=False,
                 transform=transform)
dataset = ConcatDataset([trainset, testset])
dataloader = DataLoader(dataset, 
                        batch_size=256, 
                        shuffle=True, 
                        num_workers=8)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device = }')

Using device = device(type='cpu')


In the next two cells, we are going to define our Encoder and Decoder neural networks. We will later combine these into a single neural network as an AutoEncoder.

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_size: int,
                 hidden_layers: tuple[int] | list[int],
                 dropout_rate: float=0.2,
                 activation=nn.ReLU()
                ):
        """
        The Encoder network.
        A deep neural network that learns a lower-dimensional representation of the input data by mapping it into an embedding.
        
        Parameters
        ----------
        input_size
            Number of features in the input.
        hidden_layers
            Tuple of number of neurons in the hidden layers.
        dropout_rate
            Rate at which we will perform dropout in the network.
        activation
            Specifies the activation function of the network. Defaults to ReLU.
        """
        super().__init__()

        # First layer, the input layer
        self.input_layer = torch.nn.Linear(input_size, hidden_layers[0])
        self.n_layers = 0

        ######################################################
        # Usually we could specify the layers in this way:
        # self.dense_0 = torch.nn.Linear(input_size, hidden_layers[0])
        # self.dense_1 = torch.nn.Linear(hidden_layers[0], hidden_layers[1])
        # ....
        #
        # However, instead of hardcoding this, we can do it automatically based on the hidden_layers
        # The output of one hidden_layer will always be the input for the next hidden_layer
        #######################################################
        for i in range(0, len(hidden_layers)-1):
            setattr(self, f"dense_{i}", torch.nn.Linear(hidden_layers[i],
                                                        hidden_layers[i+1])
                   )
            self.n_layers += 1

        self.activation = activation
        self.hidden_layers = hidden_layers

        # Add dropout to prevent overfitting
        self.dropout  = nn.Dropout(dropout_rate)
        self.dropout_rate = dropout_rate
        self.input_size = input_size

    def forward(self, x: Tensor) -> Tensor:
        # Special Treatment for input layer
        x = self.activation(self.input_layer(x))

        #################################################
        # forward pass through the dense layers
        # We could have written each dense layer explicitly:
        # x = self.activation(self.dense_0(x))
        # x = self.dropout(x)
        # x = self.activation(self.dense_1(x))
        # .....
        #
        # But we do it automatically:
        ##################################################
        for i in range(0, self.n_layers -1):
            x = self.activation(getattr(self, f"dense_{i}")(x))
            # dropout to prevent overfitting
            x = self.dropout(x)

        # Use layer without activation function to output embedding
        output_layer = getattr(self, f"dense_{self.n_layers-1}")
        return output_layer(x)

In [7]:
class Decoder(nn.Module):
    def __init__(self,
                 encoder: Encoder,
                 activation=nn.ReLU()
                ):
        """
        Same as the encoder, but the layers are in reverse order.
        So, we pass the encoder as input and use its hidden_sizes to specify the decoder network.
    
        Parameters
        ----------
        encoder
            This is our encoder neural network. We need to pull various attributes from our encoder object to set up our decoder network.
        activation
            Specifies the activation function of the network. Defaults to ReLU.
        """
        super().__init__()
        
        # We will set our hidden layer sizes to match what we used in the encoder.
        self.hidden_layers = encoder.hidden_layers
        n_layers = encoder.n_layers
        self.hidden_layers = self.hidden_layers[::-1]  # Reverses the order of the hidden layer sizes.

        # Reversed order -> dense_0 will be the first to apply here
        for i in range(0, n_layers):
            setattr(self, f"dense_{i}", torch.nn.Linear(self.hidden_layers[i],
                                                        self.hidden_layers[i+1])
                   )
            
        # Set the output layer to match the same size as our original input layer.
        self.output_layer = torch.nn.Linear(self.hidden_layers[-1],
                                                        encoder.input_size)
        self.n_layers = n_layers
        self.activation = activation
        self.dropout  = nn.Dropout(encoder.dropout_rate)


    def forward(self, x: Tensor) -> Tensor:
        for i in range(0, self.n_layers):
            dense_i = getattr(self, f"dense_{i}")
            x = dense_i(x)
            x = self.activation(x)
            x = self.dropout(x)
        return self.output_layer(x)

In [8]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size: int,
                 hidden_layers: tuple[int] | list[int],
                 dropout_rate: float=0.2,
                 activation=nn.ReLU()):
        """
        The complete AutoEncoder that consists of the encoder and the decoder network.
        We need this for training, but for applying the autoencoder, we will only need the encoder to map input data to an embedding.
        
        Parameters
        ----------
        input_size
            Specifies the number of features in the input.
        hidden_layers
            A tuple of number of neurons in the hidden layers used in the encoder/decoder.
        dropout_rate
            Rate at which we will perform dropout in the network.
        activation
            Specifies the activation function of the network. Defaults to ReLU.
        """
        super().__init__()
        self.encoder = Encoder(input_size, hidden_layers, dropout_rate, activation)
        self.decoder = Decoder(self.encoder, activation)
        self.hidden_layers = hidden_layers

    def forward(self, x: Tensor) -> tuple[Tensor, ...]:
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


Let's prepare our input datasets

In [9]:
X_train = trainset.data.numpy().reshape(60000, 784)
X_test = testset.data.numpy().reshape(10000, 784)
X_test.shape

(10000, 784)

In [10]:
y_train = np.array(trainset.targets)
y_test = np.array(testset.targets)

In [11]:
y = np.concatenate([y_train, y_test])
X = np.concatenate([X_train, X_test])
X.shape

(70000, 784)

Let's set up our model and display it.

In [12]:
n_input_features = X.shape[1]
# Initialize architecture of our Auto-Encoder
model = AutoEncoder(input_size=n_input_features,
                    hidden_layers=[500, 500, 2000,
                                 10 # This is the dimension of the embedding
                                 ],
                   # Prevent overfitting by deactivating 20% of the neurons during training
                    dropout_rate=0.2
                   ).to(device) # use GPU if available
print(model)

AutoEncoder(
  (encoder): Encoder(
    (input_layer): Linear(in_features=784, out_features=500, bias=True)
    (dense_0): Linear(in_features=500, out_features=500, bias=True)
    (dense_1): Linear(in_features=500, out_features=2000, bias=True)
    (dense_2): Linear(in_features=2000, out_features=10, bias=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (dense_0): Linear(in_features=10, out_features=2000, bias=True)
    (dense_1): Linear(in_features=2000, out_features=500, bias=True)
    (dense_2): Linear(in_features=500, out_features=500, bias=True)
    (output_layer): Linear(in_features=500, out_features=784, bias=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.2, inplace=False)
  )
)


Let's set our training criteria. We will use a loss function that measures the mean-squared error between the input and output of our model as we are train. We will also use stochastic gradient descent with a variable learning rate.

In [13]:
# Set the loss function
loss_ = nn.MSELoss()

# Learning Rate
lr = 0.1

# Use Stochastic Gradient Descent as optimizer with momentum 0.9
optimizer = optim.SGD(lr=lr,
                      momentum=0.9,
                      params=model.parameters())

# Reduce learning rate as training continues
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      step_size=100,
                                      gamma=0.1)

The next cell will run the training of the network.

<div class="alert alert-block alert-warning">

Warning, this may be computationally expensive.

</div>

In [ ]:
# We could restore a model to continue training from a checkpoint
#model = torch.load("./torch_models/autoencoder")

n_epochs = 300
eval_every = 10
best_loss = np.infty

# Activate training mode
model.train()

for epoch in range(n_epochs):
    losses = []
    # Iterate over data in batches
    for x_batch, y_batch in dataloader:
        # PyTorch specific; We need to reset all gradients
        optimizer.zero_grad()

        # 0. Transform input batch data from 28 X 28 to 784 features
        #   Note that our encoder maps the data into just 10 features!
        x_batch = x_batch.to(device)
        x_batch = x_batch.view(x_batch.shape[0], -1)

        # 1. Apply AutoEncoder model (forward pass).
        #    We use the output of the decoder for training.
        output = model(x_batch)[1]

        # 2. Calculate the reconstruction loss
        loss = loss_(output, x_batch)
        losses.append(loss.item())

        # 3. Backpropagate less
        loss.backward()

        # 4. Update the weights
        optimizer.step()


    mean_loss = np.round(np.mean(losses), 5)
    if (epoch+1) % eval_every == 0:
        print(f"Loss at epoch [{epoch+1} / {n_epochs}]: {mean_loss}")

    # Update learning rate as training continues
    scheduler.step()

    if mean_loss < best_loss:
        best_loss = loss
        # Store the model
        torch.save(model, "./torch_models/autoencoder")

Now that our model is trained, we can load it up for fine-tuning.

<div class="alert alert-block alert-warning">

Warning, this may be computationally expensive.

</div>

In [15]:
# Load the model
model = torch.load("./torch_models/autoencoder")

# Inference Mode for fine-tuning
model.eval()

lr = 0.1
optimizer = torch.optim.SGD(lr=lr,
                            momentum=0.9,
                            params=model.parameters()
                           )
n_epochs = 100
eval_every = 10
best_loss = np.infty

for epoch in range(n_epochs):
    losses = []
    for x_batch, y_batch in dataloader:
        # Reset gradients --> Specific for PyTorch
        optimizer.zero_grad()

        # Use GPU
        x_batch = x_batch.to(device)

        # Image has shape 28 x 28 -> Transform to 784 features using flattening
        x_batch = x_batch.view(x_batch.shape[0], -1)

        # Apply the model
        output = model(x_batch)[1]

        # Calculate the loss
        loss = loss_(output, x_batch)
        losses.append(loss.item())

        # Backpropagate the loss
        loss.backward()

        # update weights
        optimizer.step()

    mean_loss = np.round(np.mean(losses), 5)
    if (epoch+1) % eval_every == 0:
        print(f"Loss at epoch [{epoch+1} / {n_epochs}]: {mean_loss}")
    torch.save(model, "./torch_models/autoencoder-finetuned")

Loss at epoch [10 / 100]: 0.05723
Loss at epoch [20 / 100]: 0.05422
Loss at epoch [30 / 100]: 0.05199
Loss at epoch [40 / 100]: 0.05024
Loss at epoch [50 / 100]: 0.0488
Loss at epoch [60 / 100]: 0.04758
Loss at epoch [70 / 100]: 0.04651
Loss at epoch [80 / 100]: 0.04559
Loss at epoch [90 / 100]: 0.04477
Loss at epoch [100 / 100]: 0.04403


For comparison against a non-neural network method, let's use the [K-Means clustering](https://en.wikipedia.org/wiki/K-means_clustering) from Scikit-learn. 

In [32]:
from sklearn.cluster import KMeans
# Use the actual number of clusters as parameter
n_clusters = len(np.unique(y))

# Apply kmeans using sklearn
kmeans = KMeans(n_clusters=n_clusters, random_state=123)

# Get training predictions
y_pred_kmeans = kmeans.fit_predict(X)

To measure the performance of our clustering methods, we will use the [adjusted mutual information](https://en.wikipedia.org/wiki/Adjusted_mutual_information) (AMI) and [adjusted random index](https://en.wikipedia.org/wiki/Rand_index) (AMI) scores.

In [33]:
print("Accuracy of k-Means Clustering:")
ami_kmeans = adjusted_mutual_info_score(y, y_pred_kmeans)
ari_kmeans = adjusted_rand_score(y, y_pred_kmeans)
print(f"AMI: {ami_kmeans * 100 :.1f}")
print(f"ARI: {ari_kmeans * 100 :.1f}")

Accuracy of k-Means Clustering:
AMI: 49.1
ARI: 36.1


Now, let's load our pre-trained model as it was before we performed our fine-tuning and measure how our predictions compare against the K-Means clustering method.

In [34]:
model = torch.load("./torch_models/autoencoder")
X_embedded_pretrained = model(Tensor(X).to(device))[0]

In [35]:
# Apply kmeans using sklearn
kmeans = KMeans(n_clusters=n_clusters, random_state=123)

# Convert Data to CPU and apply kmeans to get the cluster predictions
y_pred_AE_pretrained = kmeans.fit_predict(X_embedded_pretrained.detach().cpu())

In [37]:
print("Accuracy for Auto-Encoder:")
ami_AE_pretrained = adjusted_mutual_info_score(y, y_pred_AE_pretrained)
ari_AE_pretrained = adjusted_rand_score(y, y_pred_AE_pretrained)
print(f"AMI: {ami_AE_pretrained * 100:.1f}")
print(f"ARI: {ari_AE_pretrained * 100:.1f}")

Accuracy for Auto-Encoder:
AMI: 51.9
ARI: 41.6


Finally, let's load our model after we performed the fine-tuning and compare its performance.

In [38]:
model = torch.load("./torch_models/autoencoder-finetuned")
X_embedded = model(Tensor(X).to(device))[0]

In [39]:
# Apply kmeans using sklearn
kmeans = KMeans(n_clusters=n_clusters, random_state=123)

# Get training predictions
y_pred_AE_finetuned = kmeans.fit_predict(X_embedded.detach().cpu())

In [31]:
print("Accuracy for Auto-Encoder:")
ami_AE_finetuned = adjusted_mutual_info_score(y, y_pred_AE_finetuned)
ari_AE_finetuned = adjusted_rand_score(y, y_pred_AE_finetuned)
print(f"AMI: {ami_AE_finetuned * 100:.1f}")
print(f"ARI: {ari_AE_finetuned * 100:.1f}")

Accuracy for Auto-Encoder:
AMI: 68.7
ARI: 62.2


We can use Pandas to put all of our model metric scores together as a convenient summary of the three methods.

In [44]:
import pandas as pd
df = pd.DataFrame({"Clustering Approach": ["k-Means", "Auto-Encoder (pre-trained)", "Auto-Encoder (fine-tuned)"],
                   "AMI": [ami_kmeans, ami_AE_pretrained, ami_AE_finetuned],
                   "ARI": [ari_kmeans, ari_AE_pretrained, ari_AE_finetuned],
                   "AMI Improvement": [None, (ami_AE_pretrained - ami_kmeans) / ami_AE_pretrained, (ami_AE_finetuned - ami_kmeans) / ami_AE_finetuned],
                   "ARI Improvement": [None, (ari_AE_pretrained - ari_kmeans) / ari_AE_pretrained, (ari_AE_finetuned - ari_kmeans) / ari_AE_finetuned]})
df["AMI"] *= 100
df["ARI"] *= 100
df["AMI Improvement"] *= 100
df["ARI Improvement"] *= 100
df["AMI"] = df["AMI"].round(1)
df["ARI"] = df["ARI"].round(1)
df["AMI Improvement"] = df["AMI Improvement"].round(1)
df["ARI Improvement"] = df["ARI Improvement"].round(1)
df

,Clustering Approach,AMI,ARI,AMI Improvement,ARI Improvement
0,k-Means,49.1,36.1,NaN,NaN
1,Auto-Encoder (pre-trained),51.9,41.6,5.3,13.2
2,Auto-Encoder (fine-tuned),68.7,62.2,28.5,42.0


Thus, while the K-Means clustering method is a very fast clustering method, we can improve our accuracy by 5.3% (13.2%) as measured by AMI (ARI) by using a neural network approach. If we fine-tune our initial neural network model we can improve our accuracy by 28.5% (42.0%) when compared to the K-Means cluster method.